# Notebook : Ingestion quotidienne de données éoliennes

## 📋 Objectif
Charger de manière incrémentale les données de production éolienne depuis GitHub vers le Lakehouse Bronze.

## 🔄 Logique
1. Identifier la date la plus récente dans Bronze
2. Calculer le jour suivant (date + 1)
3. Télécharger le fichier CSV correspondant depuis GitHub
4. Ajouter les nouvelles données en mode append

## ⚙️ Exécution
- **Fréquence recommandée** : Quotidienne (via pipeline)
- **Durée moyenne** : 50-60 secondes
- **Dépendances** : Repository GitHub public accessible

## 📦 Dépendances
- Lakehouse : LH_Wind_Power_Bronze
- Source : https://github.com/gsoulat/data-training-fabric/tree/main/eolienne

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 3, Finished, Available, Finished)

In [2]:
base_url = "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/"

bronze_table_path = "abfss://brief_dp700@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Bronze.Lakehouse/Tables/dbo/wind_power"

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 4, Finished, Available, Finished)

In [3]:
df_spark = spark.read.format("delta").load(bronze_table_path)

df_pandas = df_spark.toPandas()

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 5, Finished, Available, Finished)

In [4]:
most_recent_date = pd.to_datetime(df_pandas['date'], format ="%Y%m%d").max()

next_date = (most_recent_date + timedelta(days = 1 )).strftime("%Y%m%d")

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 6, Finished, Available, Finished)

In [5]:
print(most_recent_date)
print(next_date)

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 7, Finished, Available, Finished)

2024-06-16 00:00:00
20240617


In [6]:
file_url = f"{base_url}{next_date}_wind_power_data.csv"
print(file_url)

df_pandas_new = pd.read_csv(file_url)
df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 8, Finished, Available, Finished)

https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240617_wind_power_data.csv


In [7]:
df_spark_new = spark.createDataFrame(df_pandas_new, schema = df_spark.schema)
df_spark_new.write.format('delta').mode("append").save(bronze_table_path)


StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 9, Finished, Available, Finished)

In [8]:
print("✅ Données ajoutées avec succès dans le Lakehouse Bronze")
print(f"📊 Total après ajout : {spark.read.format('delta').load(bronze_table_path).count()} lignes")

StatementMeta(, 4f55c554-d7be-4b8e-81f8-b504ff01a850, 10, Finished, Available, Finished)

✅ Données ajoutées avec succès dans le Lakehouse Bronze
📊 Total après ajout : 1296 lignes
